连接单个设备

In [16]:
from netmiko import ConnectHandler
connection = ConnectHandler(
    host="192.168.56.102",
    username='admin',
    password='Admin@h3c.com',
    device_type='hp_comware'
)
output = connection.send_command('display arp')
print(output)
connection.disconnect()

  Type: S-Static   D-Dynamic   O-Openflow   R-Rule   M-Multiport  I-Invalid
IP address      MAC address    VLAN/VSI name Interface                Aging Type
192.168.56.100  0800-277f-9d53 1             GE1/0/1                  486   D   
192.168.56.115  0a00-2700-0013 1             GE1/0/1                  264   D   
192.168.56.103  b2e9-8008-0202 1             GE1/0/2                  763   D   
192.168.56.104  0e5b-c40f-0302 1             GE1/0/3                  111   D   


判断模块是否存在

In [14]:
from netmiko import platforms
import netmiko
platform = '\n'.join(platforms)
if 'hp_comware' in platform:
    print(f'模块存在')
else:
    print(None)

模块存在


使用python中的字典简化进行连接

In [ ]:
from netmiko import ConnectHandler
h3c_device = {
    "device_type":"hp_comware",
    "host":"192.168.56.102",
    "username":"admin",
    "password":"Admin@h3c.com"
}
connection = ConnectHandler(**h3c_device)
output = connection.send_command("display mac-address")
print(output)
connection.disconnect()

MAC Address      VLAN ID    State            Port/Nickname            Aging
0800-277f-9d53   1          Learned          GE1/0/1                  Y
0a00-2700-0013   1          Learned          GE1/0/1                  Y
0e5b-c40f-0302   1          Learned          GE1/0/3                  Y
b2e9-8008-0202   1          Learned          GE1/0/2                  Y


配置方法

In [22]:
from netmiko import ConnectHandler
h3c_device = {
    "device_type":"hp_comware",
    "host":"192.168.56.102",
    "username":"admin",
    "password":"Admin@h3c.com"
}
connection = ConnectHandler(**h3c_device)
# connection.config_mode()
output = connection.send_command('vlan 177')
# connection.exit_config_mode()
print(output)
connection.disconnect()

     ^
 % Unrecognized command found at '^' position.


getpass用法

In [ ]:
from netmiko import ConnectHandler
from getpass import getpass

password = getpass('Plaease Enter Password:')
h3c_device = {
    "device_type":"hp_comware",
    "host":"192.168.56.102",
    "username":"admin",
    "password":password
}
connection = ConnectHandler(**h3c_device)
output = connection.send_command('dis arp')
print(output)
connection.disconnect()

发送多个命令

In [ ]:
from netmiko import ConnectHandler
from getpass import getpass

password = getpass('Plaease Enter Password:')
h3c_device = {
    "device_type":"hp_comware",
    "host":"192.168.56.102",
    "username":"admin",
    "password":password
}
# config_command = ['sys','vlan 177','int vlan 177','ip add 2.2.2.2 24','desc MGT','sa f']
connection = ConnectHandler(**h3c_device)
# connection.send_config_set(config_command)
print(connection.send_command('dis vlan')+'\n')
print('-'*30)
print('执行命令第二个结果:'+'\n')
print(connection.send_command('dis ip int bri | in 2.2.2.2'))

使用netmiko连接多个设备

In [ ]:
from netmiko import ConnectHandler
import json

ip_list = ['192.168.56.102',
           '192.168.56.103',
           '192.168.56.104']

device_list = []
for ip in ip_list:
    h3c_device = {
    "device_type":"hp_comware",
    "host":ip,
    "username":"admin",
    "password":'Admin@h3c.com'
    }
    device_list.append(h3c_device)
json_formatted = json.dumps(device_list)
for each_device in device_list:
    connection = ConnectHandler(**each_device)
    print(f'Connection to {each_device["host"]}')
    output = connection.send_command('dis cu | in vlan 177')
    print(output)
    print(f"Closing Connection to {each_device["host"]}")
    connection.disconnect()

SSH上传文件_file_transfer

In [ ]:
from netmiko import ConnectHandler,file_transfer
import json

def device_connection(host):
    device = {
        'host' : host,
        'username' : 'admin',
        'password' : 'Admin@h3c.com',
        'device_type' : 'hp_comware'
        }
    connection = ConnectHandler(**device)
    return connection

def command_list(connection,commands):
    print('正在下发指令')
    output = connection.send_config_set(commands)
    print('命令下发完成')
    return output
    
def file_uploads(connection):
    src_file = input('请输入本地文件路径，例如:./config.txt:')
    dest_file = input('请输入目的文件路径，例如:flash:/:')
    file_system = input('请输入目标所属的文件系统,例如:flash或者flash:/:')
    print('开始上传文件...')
    transfer_result = file_transfer(
        connection,
        source_file = src_file,
        dest_file = dest_file,
        file_system = file_system,
        direction = "put",
        overwrite_file = True
    )
    return transfer_result

if __name__ == "__main__":
    target_host = [
        '192.168.56.102',
        '192.168.56.103',
        '192.168.56.104'
    ]
    command_lists = (
        'sftp server enable'
    )
    for connection in target_host:
        try:
            net_connect = device_connection(connection)
            cmd_output = command_list(net_connect,command_lists)
            print('命令输出预览: \n',cmd_output[:500])
            result = file_uploads(net_connect)
            print('\n--- 传输结果 ---')
            print(json.dumps(result,indent=4))

        except Exception as e:
            print(f'发生错误：{e}')

        finally:
            print("正在断开连接---")
            net_connect.disconnect()

SSH上传文件_华三(批量上传本地公钥文件，并导入设备且将公钥文件绑定目标账户)

In [ ]:
from netmiko import ConnectHandler
import os

def device_connection(host):
    """创建H3C设备连接"""
    device = {
        'host': host,
        'username': 'admin',
        'password': 'Admin@h3c.com',
        'device_type': 'hp_comware',
        'timeout': 60,  # 增加超时时间，适配设备响应慢的情况
        'session_log': None  # 如需调试，可设置为文件路径，如'netmiko_log.txt'
    }
    try:
        connection = ConnectHandler(**device)
        print(f"✅ 成功连接到设备 {host}")
        return connection
    except Exception as e:
        raise Exception(f"❌ 连接设备 {host} 失败：{str(e)}")

def execute_command(connection, commands):
    """执行多条非配置模式命令（适配H3C系统视图）"""
    print('🔧 正在下发指令')
    output = ""
    # 循环执行每条命令
    for cmd in commands:
        # 执行单条命令，拼接输出结果
        cmd_output = connection.send_command(cmd, expect_string=r'\[.*\]|<.*>')
        output += f"命令：{cmd}\n输出：{cmd_output}\n"
    print('✅ 命令下发完成')
    return output

def file_uploads(connection, host_ip):
    """通过TFTP向H3C设备上传文件（设备主动下载）"""
    # 修复路径转义问题：使用原始字符串或双反斜杠
    local_file = r'C:\Users\sys400070\.ssh\id_rsa.pub'
    remote_path = 'flash:/'
    tftp_server_ip = '192.168.56.115'
    filename = os.path.basename(local_file)  # 提取文件名
    
    # 拼接完整的TFTP下载命令（指定目标路径）
    upload_cmd = f"tftp {tftp_server_ip} get {filename} {remote_path}{filename}"
    print(f'📤 开始从TFTP服务器 {tftp_server_ip} 下载文件 {filename} 到设备 {host_ip}...')
    
    # 修复expect_string：匹配H3C的用户视图<*>或系统视图[*]
    output = connection.send_command(upload_cmd, expect_string=r'\[.*\]|<.*>')
    
    # 增加结果校验
    if "成功" in output or "已完成" in output or "Success" in output:
        print(f"✅ 设备 {host_ip} 文件上传成功！")
    else:
        print(f"❌ 设备 {host_ip} 文件上传失败，输出：{output}")
    return output

if __name__ == "__main__":
    # 目标设备列表
    target_hosts = [
        '192.168.56.102',
        '192.168.56.103',
        '192.168.56.104'
    ]
    # 要执行的多条系统视图命令（列表格式）
    command = [
        'sys',
        'public-key peer user01_key import sshkey flash:/id_rsa.pub',
        'ssh user admin service-type  stelnet authentication-type  publickey assign publickey user01_key'
    ]

    # 修复循环逻辑：重命名循环变量，避免重名
    for host in target_hosts:
        net_connect = None  # 初始化连接对象，避免finally中未定义
        try:
            # 1. 连接设备
            net_connect = device_connection(host)
            
            # 2. 执行多条命令
            cmd_output = execute_command(net_connect, command)
            print('📝 命令输出预览: \n', cmd_output[:500])
            
            # 3. 执行文件上传
            # transfer_result = file_uploads(net_connect, host)
            # print('\n--- 传输结果 ---')
            # print(transfer_result)  # 直接打印字符串，无需JSON序列化

        except Exception as e:
            print(f'❌ 处理设备 {host} 时发生错误：{e}')

        finally:
            # 确保连接对象存在时才断开
            if net_connect:
                print(f"🔌 正在断开与 {host} 的连接---")
                net_connect.disconnect()
            print("-" * 50)  # 分隔符，优化输出可读性